read in csv input file data

In [120]:
def read_data(file_name):
  df = pd.read_csv(file_name)
  df = df.fillna('')
  df.head(15)

  initialize_machine(df)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

turn df into usable data structures for machine and tapes

In [143]:
def initialize_machine(df):
  #initialize values
  machine_name = df.iloc[0,0]
  k_value = int(df.iloc[0,1])
  start_state = df.iloc[2,0]
  accept_state = df.iloc[3,0]
  reject_state = df.iloc[4,0]
  halt_condition = int(df.iloc[5,1])

  tapes = []
  transitions = {}

  #put tapes in a list
  for i in range(k_value):
    tapes.append(df.iloc[1, i])

  #get rest of transitions, put into dict
  num_transitions = len(df[6:])
  for j in range(6, num_transitions+6):
    init_state = df.iloc[j,0]
    next_state = df.iloc[j,k_value+1]
    input_symbols = tuple(df.iloc[j,1:k_value+1])
    new_symbols = tuple(df.iloc[j,k_value+2:k_value+k_value+2])
    moves = tuple(df.iloc[j, k_value+k_value+2:])

    transitions[(init_state, input_symbols)] = (next_state, new_symbols, moves)
  """
  print(transitions)
  print(accept_state)
  print(reject_state)
  print(halt_condition)
  print(tapes)
  """
  val = move_through_machine(machine_name, tapes, transitions, accept_state, reject_state, halt_condition, k_value)

move through machine

In [144]:
def move_through_machine(machine_name, tapes, transitions, accept_state, reject_state, halt_condition, k_value):
  #initialize tape heads at first char as list
  heads = [0] * k_value

  start_state = list(transitions.keys())[0][0] #assume first listed transition state is start state
  current_state = start_state
  #print(current_state)

  print(f"Machine: {machine_name}")
  print(f"Tape Input(s): {tapes}")
  print()

  step = 1
  while step < halt_condition:
    transition_found = False
    print(f"Step {step}:")

    #print state of all tapes, with space before where tape heads are
    for i in range(k_value):
      print(f"Tape {i+1}:", end=" ")
      head_spot = heads[i]
      print(f"{tapes[i][:head_spot]} {tapes[i][head_spot:]}")

    #find symbols at tape heads rn
    current_symbols = [tapes[i][heads[i]] for i in range(k_value)]
    current_symbols = tuple(current_symbols)
    #print(current_symbols)

    #find appropriate transition
    #print(transitions)
    #print(current_state)
    acceptable_symbols = []
    trans_states = [list(transitions.keys())[i][0] for i in range(len(transitions))]
    #print(trans)
    #print(transitions[(current_state, current_symbols)])
    if current_state in trans_states: #if current state even in list of initial states
      match_key = None #store matching key if found, needed bc of wild card

      if (current_state, current_symbols) in transitions.keys():
        #print('hey')
        #print(current_state)
        #print(current_symbols)
        #print(transitions[(current_state, current_symbols)])
        next_state, new_symbols, moves = transitions[(current_state, current_symbols)]
        print(f"Selected Transition: {(current_state, current_symbols)} -> {next_state}, {new_symbols}, {moves}")
        transition_found = True
        print()

      else:
        allow = 1 #sees if * allows passing
        for (start_state, symbols) in transitions.keys():
          if start_state == current_state:
              match = True
              for i in range(len(symbols)):  #iterate through indices
                  if symbols[i] != current_symbols[i] and symbols[i] != '*':
                      match = False
                      break
              if match:
                  best_match_key = (start_state, symbols)
                  break
        if best_match_key:
          #access the transition using the found key
          next_state, new_symbols, moves = transitions[best_match_key]
          print(f"Selected Transition: {best_match_key} -> {next_state}, {new_symbols}, {moves}")
          transition_found = True
          print()
        else:
          #if no matching transition is found, halt machine
          print("No valid transition found. Halting.")
          break
    else: #nowhere to go from current state
      print('halt')
      break

    if transition_found: #overwrite necessary characters
      for i in range(k_value):
        if new_symbols[i] != '*':
          tapes[i] = tapes[i][:heads[i]] + new_symbols[i] + tapes[i][heads[i]+1:]

    current_state = next_state
    current_symbols = new_symbols
    for index, move in enumerate(moves):
      if move == 'R' and heads[index]+1 < len(tapes[index]):
        heads[index] = (heads[index] + 1)
      elif move == 'R' and heads[index] == len(tapes[index])-1:
        tapes[index] += '_'
        heads[index] = (heads[index] + 1)
      elif move == 'L' and heads[index] > 0:
        heads[index] = (heads[index] - 1)

    if current_state == 'qacc':
      print("Machine Accepted")
      return 0
    elif current_state == 'qrej':
      print("Machine Rejected")
      return 1
    else:
      step += 1

  print("Halting condition met. Halting")
  return 1


In [152]:
import pandas as pd

#designate input file
file_name = "/content/theory - simple machine.csv"
#file_name = "/content/theory - DNA complementer.csv"
#file_name = "/content/theory - palindrome checker.csv"


#effectively the "main", just call this and it will output(for clean output)
read_data(file_name)



Machine: Machine One
Tape Input(s): ['110', '101']

Step 1:
Tape 1:  110
Tape 2:  101
Selected Transition: ('q0', ('*', '*')) -> q1, ('1', '*'), ('R', 'S')

Step 2:
Tape 1: 1 10
Tape 2:  101
Selected Transition: ('q1', ('1', '1')) -> q2, ('0', '0'), ('R', 'R')

Step 3:
Tape 1: 10 0
Tape 2: 0 01
Selected Transition: ('q2', ('0', '0')) -> qacc, ('*', '*'), ('R', 'R')

Machine Accepted
